# コードジェネレーター

要件：フロンティアモデルを使用して、Pythonコードから高性能C++コードを生成する


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">リマインダー: 最新コードを取得してください</h2>
            <span style="color:#f71;">これらのラボは継続的に改善しており、例題や演習を追加しています。毎週初めに、最新のコードを使用していることを確認してください。<br/>
            まず、<a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull を実行し、必要に応じて変更をマージしてください</a>。何か問題がありましたら、ChatGPT にマージ方法を尋ねてみるか、私にご連絡ください。<br/><br/>
            コードを pull した後、llm_engineering ディレクトリから Anaconda プロンプト (PC) またはターミナル (Mac) で次のコマンドを実行してください。:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Anaconda ではなく virtualenv を使用した場合は、Powershell (PC) またはターミナル (Mac) で、有効化した環境から次のコマンドを実行してください。:<br/>
            <code>pip install -r requirements.txt</code>
            <br/>変更を反映させるため、カーネルを再起動してください (「カーネル」メニュー >> 「カーネルを再起動してすべてのセルの出力をクリア」)。
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">重要事項</h1>
            <span style="color:#900;">
            このラボでは、GPT-4oとClaude-3.5-Sonnetを使用しています。これらは少し高価なモデルです。それでもコストは低いですが、コストを極限まで抑えたい場合は、推奨されているモデル（ここから3セル下）への切り替えを行ってください。
            </span>
        </td>
    </tr>
</table>

In [1]:
# import

# 基本
import os
import io
import sys
import subprocess

# WebAPI+Key
from dotenv import load_dotenv
from openai import OpenAI
#import google.generativeai
#import anthropic

# 出力、WebUI
from IPython.display import Markdown, display, update_display
import gradio as gr

In [2]:
# 環境

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
#os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [3]:
# 初期化 - 最後の2行は低コストモデル

openai = OpenAI()
#claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4.1" # "gpt-4o" # コーディングが得意な新モデル「GPT-4.1」
#CLAUDE_MODEL = "claude-3-5-sonnet-20240620" # コーディングが、かなり得意らしい。

# コストを非常に低く保ちたいですか？
#（生成コードでコンパイルエラー出るので高コストを使ったほうが良いかも。
# OPENAI_MODEL = "gpt-4o-mini"
# CLAUDE_MODEL = "claude-3-haiku-20240307"

In [4]:
# system_message
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Ubuntu. \
Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. \
The C++ response needs to produce an identical output in the fastest possible time."
# "あなたは、Ubuntu 向けに高性能 C++ で Python コードを再実装するアシスタントです。\
# C++ コードのみで応答してください。コメントは控えめに使用し、時折コメントする以外は説明は提供しないでください。\
# C++ の応答では、可能な限り最短時間で同一の出力を生成する必要があります。"

In [5]:
# user_prompt
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. \
    Respond only with C++ code; do not explain your work other than a few comments. \
    Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    # "このPythonコードを、最短時間で同一の出力を生成する、可能な限り高速な実装でC++に書き直してください。\
    # C++コードのみで回答してください。作業内容については、コメント以外に説明は不要です。\
    # intオーバーフローが発生しないように、数値型に注意してください。iomanipなどの必要なC++パッケージはすべて#includeを忘れないでください。\n\n"
    user_prompt += python
    return user_prompt

In [6]:
# messages = system_message + user_prompt_for(x)
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
# *.cppファイルに書き込みます
def write_output(cpp, name):
    code = cpp.replace("```cpp","").replace("```","")
    with open(name + ".cpp", "w") as f:
        f.write(code)

In [8]:
# GPTで最適化された実装（C++）に変換 
def optimize_gpt(python, name):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply, "optimize_gpt_" + name)

In [ ]:
# Claudeで最適化された実装（C++）に変換
def optimize_claude(python, name):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply, "optimize_claude_" + name)

In [9]:
# πの近似計算（数値級数）
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

上記の関数は、実は有名な **π（パイ）を近似する計算**の一種

具体的には、次のような級数展開に近い形：

$$
\pi = 4 \left(1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \dots\right)
$$

ただしこのコードでは、以下の形に似ています：
$$
\sum_{i=1}^{\infty} \left( \frac{1}{4i - 3} - \frac{1}{4i - 1} \right)
$$

In [10]:
# pythonを実行（危険なのでユーザ入力を持っていかないように）
exec(pi)

Result: 3.141592658589
Execution Time: 24.583790 seconds


In [11]:
# GPTでPythonをC++に変換してファイルに保存
optimize_gpt(pi, "pi")

#include <iostream>
#include <iomanip>
#include <chrono>

int main() {
    using namespace std;
    using namespace chrono;

    const int iterations = 100000000;
    const int param1 = 4;
    const int param2 = 1;
    double result = 1.0;

    auto start = high_resolution_clock::now();

    for (int i = 1; i <= iterations; ++i) {
        double j1 = static_cast<double>(i) * param1 - param2;
        result -= (1.0 / j1);
        double j2 = static_cast<double>(i) * param1 + param2;
        result += (1.0 / j2);
    }

    result *= 4.0;

    auto end = high_resolution_clock::now();
    double elapsed = duration_cast<duration<double>>(end - start).count();

    cout << fixed << setprecision(12);
    cout << "Result: " << result << endl;
    cout << "Execution Time: " << setprecision(6) << elapsed << " seconds" << endl;
    return 0;
}

# C++をコンパイルして実行
この次のセルには、Ubuntu に C++ ファイルをコンパイルするコマンドが含まれています。  
`optimized.cpp`を`optimized`と呼ばれる実行可能ファイルにコンパイルします  
次に`optimized`と呼ばれるプログラムを実行します。

次のラボ（4日目）では、学生がMac、PC、Linuxの効率的なコードにコンパイルする完全なソリューションを提供しました！

これを待つことができます。または、プラットフォームでこれを行う方法についてGoogle（またはChatGpt！）を使用してから、以下の行を交換できます。
このステップに慣れていない場合は、確かにスキップできます。Macでどのように機能するかを正確に示します。

代わりに：学生Sandeep K.G. PythonとC++コードをオンラインで実行して、そのようにテストできることを指摘しています。ありがとうSandeep！ 

> 正確な比較ではありませんが、パフォーマンスの違いのアイデアを得ることができます。  
> Online C++ compilerの例: https://www.programiz.com/cpp-programming/online-compiler/

In [12]:
# C++ファイルをコンパイルし、実行可能ファイルを実行

# C++ファイルをコンパイル
!g++ -O2 -o optimize_gpt_pi optimize_gpt_pi.cpp
# コンパイルした実行ファイルを実行
!./optimize_gpt_pi

Result: 3.141592658589
Execution Time: 0.252126 seconds


In [13]:
# 擬似乱数を使って生成された整数配列に対し、最大部分配列和（Maximum Subarray Sum）を複数回（20回）計算し、その合計を出力する。

# lcg：線形合同法により、一定の周期で乱数を生成する擬似乱数生成器
# max_subarray_sum：lcgで長さnの乱数配列を生成し最大部分配列和を求める
# total_max_subarray_sum：長さ10000の乱数配列から最大部分配列和を求める処理を20回繰り返し合計

python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [14]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 106.575778 seconds


In [16]:
# GPTでPythonをC++に変換してファイルに保存
optimize_gpt(python_hard, "ph")

```cpp
#include <iostream>
#include <vector>
#include <cstdint>
#include <chrono>
#include <iomanip>
#include <limits>

// Fast LCG generator
class LCG {
public:
    using uint32 = uint32_t;
    static constexpr uint32 a = 1664525;
    static constexpr uint32 c = 1013904223;
    static constexpr uint64_t m = uint64_t(1) << 32;

    explicit LCG(uint32 seed): value(seed) {}

    uint32 next() {
        value = a * value + c;
        return value;
    }

private:
    uint32 value;
};

int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> arr;
    arr.reserve(n);
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        arr.push_back(static_cast<int>(lcg.next() % range) + min_val);
    }
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    // Brute-force as in the python code
    for (int i = 0; i < n; ++i) {
        int64_t curr_sum = 0;
        for (int j = i; j < n; ++j) {
            curr_sum

In [17]:
# C++ファイルをコンパイルし、実行可能ファイルを実行

# C++ファイルをコンパイル
!g++ -O2 -o optimize_gpt_ph optimize_gpt_ph.cpp
# コンパイルした実行ファイルを実行
!./optimize_gpt_ph

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.811339 seconds


# Claude

In [ ]:
# ClaudeでPythonをC++に変換してファイルに保存
optimize_claude(pi, "pi")

In [ ]:
# Claudeのために繰り返し - 繰り返しますが、プラットフォームに適切なアプローチを使用してください

# C++ファイルをコンパイル
!g++ -O2 -o optimize_gpt_pi optimize_gpt_pi.cpp
# コンパイルした実行ファイルを実行
!./optimize_gpt_pi

In [ ]:
# ClaudeでPythonをC++に変換してファイルに保存
optimize_claude(python_hard, "ph")

In [ ]:
# Claudeのために繰り返し - 繰り返しますが、プラットフォームに適切なアプローチを使用してください

# C++ファイルをコンパイル
!g++ -O2 -o optimize_gpt_ph optimize_gpt_ph.cpp
# コンパイルした実行ファイルを実行
!./optimize_gpt_ph

# Gradio

In [18]:
import gradio as gr

In [19]:
# optimize_gptのGUI版
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
# optimize_claudeのGUI版
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [20]:
# Gradioから実行するコントローラ
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    #elif model=="Claude":
    #    result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [21]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


gio: http://127.0.0.1:7868/: Operation not supported


In [22]:
# 入力されたpythonコードを実行する
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code) # 危険（codeに攻撃者の入力が渡らないように注意
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [23]:
# プラットフォームのC++コードをコンパイルするために、トライブロックのコードを変更
# 最適化されたC++コードをコンパイルおよび実行するUbuntuバージョン：
def execute_cpp(code):
    code_name = "optimize_x_pg"
    write_output(code, code_name)
    try:
        # M1 Mac
        #compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimize_x_pg", "optimize_x_pg.cpp"]
        # Ubuntu
        compile_cmd = ["g++", "-O2", "-o", code_name, code_name + ".cpp"]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        run_cmd = ["./" + code_name]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [24]:
css = """
.python {background-color: # 306998;}
.cpp {background-color: # 050;}
"""

In [25]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


gio: http://127.0.0.1:7869/: Operation not supported
